In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

### Load MNIST Data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Start TensorFlow InteractiveSession

Tensorflow relies on a highly efficient C++ backend to do its computation. The connection to this backend is called a session. The common usage for TensorFlow programs is to first create a graph and then launch it in a session.

In [3]:
sess = tf.InteractiveSession()

### Build a Softmax Regression Model

In this section we will build a softmax regression model with a single linear layer. In the next section, we will extend this to the case of softmax regression with a multilayer convolutional network.

#### Placeholders

We start building the computation graph by creating nodes for the input images and target output classes.

In [4]:
features = tf.placeholder(tf.float32, shape=[None, 784])
labels = tf.placeholder(tf.float32, shape=[None, 10])

#### Variables

We now define the weights W and biases b for our model. We could imagine treating these like additional inputs, but TensorFlow has an even better way to handle them: Variable. A Variable is a value that lives in TensorFlow's computation graph. It can be used and even modified by the computation. In machine learning applications, one generally has the model parameters be Variables.

In [5]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

We pass the initial value for each parameter in the call to tf.Variable. In this case, we initialize both W and b as tensors full of zeros. W is a 784x10 matrix (because we have 784 input features and 10 outputs) and b is a 10-dimensional vector (because we have 10 classes).

Before Variables can be used within a session, they must be initialized using that session. This step takes the initial values (in this case tensors full of zeros) that have already been specified, and assigns them to each Variable. This can be done for all Variables at once.

In [6]:
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


#### Predicted Class and Cost Function

We can now implement our regression model. It only takes one line! We multiply the vectorized input images x by the weight matrix W, add the bias b, and compute the softmax probabilities that are assigned to each class.

In [7]:
predictions = tf.nn.softmax(tf.matmul(features,W) + b)

The cost function to be minimized during training can be specified just as easily. Our cost function will be the cross-entropy between the target and the model's prediction.

In [8]:
cross_entropy = -tf.reduce_sum(labels*tf.log(predictions))

Note that `tf.reduce_sum` sums across all images in the minibatch, as well as all classes. We are computing the cross entropy for the entire minibatch.

### Train the Model

Now that we have defined our model and training cost function, it is straightforward to train using TensorFlow. Because TensorFlow knows the entire computation graph, it can use automatic differentiation to find the gradients of the cost with respect to each of the variables. TensorFlow has a variety of builtin optimization algorithms. For this example, we will use steepest gradient descent, with a step length of 0.01, to descend the cross entropy.

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

TensorFlow just added new operations to the computation graph. These operations included ones to compute gradients, compute parameter update steps, and apply update steps to the parameters.

The returned operation `train_step`, when run, will apply the gradient descent updates to the parameters. Training the model can therefore be accomplished by repeatedly running `train_step`.

In [10]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={features: batch[0], labels: batch[1]})

Each training iteration we load 50 training examples. We then run the `train_step` operation, using `feed_dict` to replace the placeholder tensors `features` and `labels` with the training examples. Note that you can replace any tensor in your computation graph using feed_dict -- it's not restricted to just placeholders.

### Evaluate the Model

First we'll figure out where we predicted the correct label. `tf.argmax` gives you the index of the highest entry in a tensor along some axis. For example, `tf.argmax(predictions, 1)` is the label our model thinks is most likely for each input, while `tf.argmax(labels, 1)` is the true label. We can use `tf.equal` to check if our prediction matches the truth.

In [11]:
correct_prediction = tf.equal(tf.argmax(predictions,1), tf.argmax(labels,1))

In [12]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
print(accuracy.eval(feed_dict={features: mnist.test.images, labels: mnist.test.labels}))

0.9069


### Build a Multilayer Convolutional Network

Getting 91% accuracy on MNIST is bad. It's almost embarrassingly bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: a small convolutional neural network. This will get us to around 99.2% accuracy -- not state of the art, but respectable.

#### Weight Initialization

To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons." Instead of doing this repeatedly while we build the model, let's create two handy functions to do it for us.

In [14]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#### Convolution and Pooling

TensorFlow also gives a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

In [15]:
def conv2d(features, W):
    return tf.nn.conv2d(features, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(features):
    return tf.nn.max_pool(features, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#### First Convolutional Layer

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolutional will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

In [16]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, we first reshape the features to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

In [17]:
x_image = tf.reshape(features, [-1,28,28,1])

In [18]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#### Second Convolutional Layer

In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

In [19]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#### Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [20]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#### Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.

In [21]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#### Readout Layer

Finally, we add a softmax layer, just like for the one layer softmax regression above.

In [22]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

predictions=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### Train and Evaluate the Model

To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network above. The differences are that: we will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer; we will include the additional parameter `keep_prob` in `feed_dict` to control the dropout rate; and we will add logging to every 100th iteration in the training process.

In [23]:
cross_entropy = -tf.reduce_sum(labels*tf.log(predictions))

In [24]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [25]:
correct_prediction = tf.equal(tf.argmax(predictions,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={features:batch[0], labels: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={features: batch[0], labels: batch[1], keep_prob: 0.5})
print("test accuracy %g"%accuracy.eval(feed_dict={features: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}))

#### MINST Dataset

https://www.google.com/search?q=mnist+dataset&source=lnms&tbm=isch&sa=X&ved=0ahUKEwjts9rn0NXWAhWmrVQKHRppCn4Q_AUICigB&biw=1661&bih=836

The Unreasonable Effectiveness of Recurrent Neural Networks
Musings of a Computer Scientist.

http://karpathy.github.io/2015/05/21/rnn-effectiveness/